In [3]:
import json

In [9]:
with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/train_predictions.json', 'r', encoding='utf-8') as f:
    data_1 = json.load(f)

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/train_predictions_2.json', 'r', encoding='utf-8') as f:
    data_2 = json.load(f)

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/train_predictions_3.json', 'r', encoding='utf-8') as f:
    data_3 = json.load(f)

list1 = data_1['predictions']
list2 = data_2['predictions']
list3 = data_3['predictions']


print(len(list1), len(list2), len(list3))
print(type(list1), type(list2), type(list3))
merged_list = list1 + list2 + list3

print(len(merged_list))

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/train_predictions_merged.json', 'w', encoding='utf-8') as f:
    json.dump(merged_list, f, ensure_ascii=False, indent=4)

print("Merged predictions saved successfully.")

96 7 1
<class 'list'> <class 'list'> <class 'list'>
104
Merged predictions saved successfully.


In [4]:
import json
# 通过检索里面的内容，看是否相同，从而更改对应的sent_id，并让顺序进行一个对齐。

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/train_predictions_merged.json', 'r', encoding='utf-8') as f:
    merged_data = json.load(f)

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/datas/train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

data = []

for item in train_data:
    sent = item['sent']
    sent_id = item['sent_id']

    for merged_item in merged_data:
        if merged_item['sent'] == sent:
            merged_item['sent_id'] = sent_id
            data.append(merged_item)
            break

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/train_predictions_merged_aligned.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Aligned predictions saved successfully.")

Aligned predictions saved successfully.


In [5]:
# 合并train和qwen256b出来的标签和概率。

import json

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/datas/train.json', 'r', encoding='utf-8') as f:
    origin_train = json.load(f)

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/train_predictions_merged_aligned.json', 'r', encoding='utf-8') as f:
    qwen256_train = json.load(f)


merge_train = []

for i in range(len(origin_train)):
    item = origin_train[i]
    item_qwen256 = qwen256_train[i]
    if item["sent"] != item_qwen256["sent"]:
        print(item["sent"])
        print(item["sent_id"])
    data_dict={
        "sent_id": item["sent_id"],
        "sent": item["sent"],
        "CourseGrainedErrorType": item["CourseGrainedErrorType"],
        "FineGrainedErrorType": item["FineGrainedErrorType"],
        "CourseGrainedErrorType_qwen256": item_qwen256["CourseGrainedErrorType"],
        "FineGrainedErrorType_qwen256": item_qwen256["FineGrainedErrorType"],
        "coarse_probabilities_qwen256": item_qwen256["coarse_probabilities"],
        "fine_probabilities_qwen256": item_qwen256["fine_probabilities"]
    }

    merge_train.append(data_dict)

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/datas/train_merged_origin+qwen256.json', 'w', encoding='utf-8') as f:
    json.dump(merge_train, f, ensure_ascii=False, indent=4)

print("train_merged_origin+qwen256.json saved.")

train_merged_origin+qwen256.json saved.


In [6]:
import json

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/datas/train.json', 'r', encoding='utf-8') as f:
    origin_train = json.load(f)

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/train_predictions_merged_aligned.json', 'r', encoding='utf-8') as f:
    qwen256_train = json.load(f)


for i in range(len(origin_train)):
    sent_1 = origin_train[i]['sent']
    flag = False
    for j in range(len(qwen256_train)):
        sent_2 = qwen256_train[j]['sent']
        if sent_1 == sent_2:
            flag = True
    
    if flag == False:
        print(origin_train[i]['sent_id'])
        print(sent_1)
            

In [3]:
import json

# 读取数据文件
with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/val_result.json', 'r', encoding='utf-8') as f:
    data_predict = json.load(f)

with open('/mnt/cfs/huangzhiwei/NLP-WED0910/datas/val.json', 'r', encoding='utf-8') as f:
    data_label = json.load(f)

# 创建句子到sent_id的映射字典，提高查找效率
sent_to_id = {item['sent']: item['sent_id'] for item in data_label}

# 统计匹配情况
matched_count = 0
total_count = len(data_predict['predictions'])

# 更新predictions中的sent_id
for data in data_predict['predictions']:
    sent = data['sent']
    if sent in sent_to_id:
        data['sent_id'] = sent_to_id[sent]
        matched_count += 1
    else:
        print(f"警告: 未找到匹配的句子: {sent[:50]}...")  # 只显示前50个字符
        # 可以选择设置默认值或跳过
        # data['sent_id'] = None

# 保存结果
with open('/mnt/cfs/huangzhiwei/NLP-WED0910/qwen3-256b-data/val_result_update.json', 'w', encoding='utf-8') as f:
    json.dump(data_predict, f, ensure_ascii=False, indent=4)

print(f"验证预测结果已成功更新sent_id")
print(f"匹配统计: {matched_count}/{total_count} ({matched_count/total_count*100:.1f}%)")

验证预测结果已成功更新sent_id
匹配统计: 27/27 (100.0%)
